In [31]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2

In [32]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=2, n_repeated=0, n_classes=2, n_clusters_per_class=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [33]:
pipe = Pipeline([('reduc_dim', PCA()), ('select_k_Best', SelectKBest()), ('svc', SVC())])

In [36]:
n_comp = [7, 8, 9]
k_features = [3 , 5, 8]
C_OPTIONS = [0.1, 0.5, 1]

param_grid = {'reduc_dim__n_components': n_comp,'select_k_Best__k': k_features, 'svc__C': C_OPTIONS}

In [ ]:
grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid, verbose = 2)
grid.fit(X_train, y_train)

{'reduc_dim__n_components': 9, 'select_k_Best__k': 8, 'svc__C': 1}

In [4]:
# pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
# # The pipeline can be used as any other estimator
# # and avoids leaking the test set into the train set
# pipe.fit(X_train, y_train)

# pipe.score(X_test, y_test)

0.908

In [ ]:
# parametes du scaler
# parametres du SVC

In [ ]:
# C_OPTIONS = [0.1, 0.5, 1]

# param_grid = [
#     {
#         'scaler__with_mean':['True', 'False'], 
#     },
#    {

#         'svc__C': C_OPTIONS
#     },
# ]

Column transformer

In [39]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

# Load data from https://www.openml.org/d/40945
X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

In [42]:
X.head(2)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


In [47]:
X.shape

(1309, 13)

In [45]:
X.isnull().sum()

pclass          0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [46]:
X.dtypes

pclass        float64
name           object
sex          category
age           float64
sibsp         float64
parch         float64
ticket         object
fare          float64
cabin          object
embarked     category
boat           object
body          float64
home.dest      object
dtype: object

In [53]:
numeric_features = ['age', 'fare']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())])

In [54]:
categorical_features = ['embarked', 'sex', 'pclass']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [55]:
preprocessor = ColumnTransformer(
    transformers=[
        ('nummeric_tf', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [56]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocesseur', preprocessor),
                      ('classifier', LogisticRegression())])

In [62]:
num_imput_strat = ['median', 'mean']
C_options = [0.1, 0.5, 1.0]

param_grid = {'preprocesseur__nummeric_tf__imputer__strategy': num_imput_strat, 'classifier__C': C_options}

In [63]:
# grid = GridSearchCV(clf, n_jobs=1, param_grid=param_grid, verbose = 2, cv=2)
# grid.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] classifier__C=0.1, preprocesseur__nummeric_tf__imputer__strategy=median 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: ignored

In [61]:
grid.best_params_

{'classifier__C': 0.1, 'preprocesseur__num__imputer__strategy': 'median'}

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))